**几个概念：**

>    并发：一个时间段内，有几个程序在同一个CPU上运行，但是任意时刻只有一个程序在CPU上运行。
>
>    并行：在任意时刻点上，有多个程序同时运行在多个CPU上。并行的数量和CPU的数量是一致的。
>
>    同步：代码调用IO操作时，必须等待IO操作完成才返回的调用方式。
>
>    异步：代码调用IO操作时，不必等IO操作完成就返回的调用方式。
>
>    阻塞：是指调用函数的额时候当前线程被挂起。
>
>    非阻塞：是指调用函数的时候当前线程不会被挂起，而是立即返回。
    
**什么是协程？**
> 协程就是可以暂停的函数，或者说有多个入口的函数。可以向暂停的地方传入值的函数。

## 1、生成器基础

### 不仅可以产出值，还可以接收值

In [7]:
def gen_func():
    # 这行代码的作用
    # 1、可以产出值 2、可以接收调用方传递进来的值
    html = yield 'http://www.baidu.com'
    print('这是传递进来的值： ' + html)
    yield 2
    yield 3
    return 'ysir'

In [12]:
1、生成器的next()方法和send()方法
gen = gen_func()
# 启动生成器的方式有两种，next()和send()
# url = next(gen)
url = gen.send(None) # 在调用send发送非none值之前，我们必须调用一次生成器。两种方式：1、gen.send(None) 2、next(gen)
print('这是生成器产出的值： ' + url)
html = 'ysir'
print(gen.send(html)) # send方法可以传递值到生成器内部，同时还可以重启生成器执行至下一个yield，相当于传递完值之后执行了next()
print(gen.send(None))

这是生成器产出的值： http://www.baidu.com
这是传递进来的值： ysir
2
3


In [14]:
# 2、生成器的close()方法
gen = gen_func()
next(gen)
gen.close()

In [17]:
# 3、生成器的throw()方法
gen = gen_func()
print(next(gen))
# throw()的第一个异常，对应的是第一个yield语句，可以在第一个yield中捕获了该异常
gen.throw(Exception, 'download error')
print(next(gen))

http://www.baidu.com


Exception: download error

## 2、生成器yield from
python 3.3版本新加的语法

### 2.1 简单示例

In [24]:
from itertools import chain

# chain可以将可迭代对象连接起来做for循环
my_list = [1,2,3]
my_dict = {'ysir 1': 'url 1', 'ysir 2': 'url 2', 'ysir 3': 'url 3'}

for value in chain(my_list, my_dict, range(5, 10)):
    print(value)

1
2
3
ysir 1
ysir 2
ysir 3
5
6
7
8
9


In [23]:
# 自己模拟实现上面的chain()
def mychain(*args):
    for my_iterable in args:
        yield from my_iterable
        
for value in mychain([1,2,3,['a','b']], range(5,10)):
    print(value)

1
2
3
['a', 'b']
5
6
7
8
9


### 2.2 yield from进阶

yield会在调用方（main函数）与子生成器（gen）之间通过委托生成器（g1）建立一个**双向**通道

In [25]:
def g1(gen):
    yield from gen
    
def main():
    g = g1()
    g.send(None)

